# Caracterization of 2-qubit states

We are hoping to caracterize the families of states of the format
$$\rho_{AB}^{\eta} = \eta\rho_{AB}+(1-\eta)\mathbb{I}.$$

I will be creating the target state ($\rho_{AB}$) either by the Bures metric or the Hilbert-Schmidt metric (this part is already finalized).

The tests we are planning on doing is:
- Entanglement
    - PPT test (finding the $\eta_{\text{ent}}$, for which the family begins to be entangled)
        - This part is already finalized.
- Bell's non-locality 
    - Lower bound: SDP
    - Upper bound: Bell's inequalities
- Steering 
    - Upper bound: Sew Saw
    - Lower bound: SDP
        - This part is already finalized.

![Caracterization of the states](caracterization.png)

In [25]:
# Importing the necessary libraries

import numpy as np
from numpy import linalg as LA
from random import random
from scipy.stats import unitary_group
import picos as pic

In [3]:
# Functions to create the target state

# Generation of the random matrix from the Ginibre ensemble
def G_matrix(m,n):
    # Matrix G of size m x n
    G = np.zeros((m,n),dtype=np.complex_)
    for k in range(m):
        for l in range(n):
            G[k,l] = random()+random()*1j
    return G

# Generation a random mixed density matrix (Bures metric)
def rho_mixed(n):
    # n = dimension of the state    
    # Create random unitary matrix
    U = unitary_group.rvs(n)
    # Create random Ginibre matrix
    G = G_matrix(n,n)
    # Create identity matrix
    I = np.eye(4)
    # Construct density matrix
    rho = (I+U)*G*(G.conjugate().T)*(I+U.conjugate().T)
    # Normalize density matrix
    rho = rho/(rho.trace())
    return rho

# Generation a random mixed density matrix (Hilbert-Schmidt metric)
def rho_mixed_HS(n):
    # n = dimension of the state  
    # Create random Ginibre matrix
    G = G_matrix(n,n)
    # Construct density matrix
    rho = G*(G.conjugate().T)
    # Normalize density matrix
    rho = rho/(rho.trace())
    return rho

In [4]:
# Generating the target state (Werner state)

#Create the base kets |00>, |01>, |10> and |11>
ket_00 = np.array([[1],[0],[0],[0]])
ket_01 = np.array([[0],[1],[0],[0]])
ket_10 = np.array([[0],[0],[1],[0]])
ket_11 = np.array([[0],[0],[0],[1]])
#Create |psi> = (|01>-|10>)/sqrt(2) and rho = |psi><psi|
psi = (ket_01-ket_10)/np.sqrt(2)
rho_AB = psi@np.transpose(psi)

# Generating the separable state

rho_I = np.eye(4)/4

## Entanglement

The function `Ent_cert(rho)` will say if the state is entangled or not, by calculating the partial transpose of the state and checking if it is positive.

Here we find the $\eta_{\text{ent}}$ in a precision of 


In [5]:
# Defining the function that certifies entanglement

def Ent_cert(rho):
    # Calculate partial transpose
    n = rho.shape
    rho_TA = np.zeros((n[0],n[1]),dtype=np.complex_)
    a = int(n[0]/2)
    b = int(n[1]/2)
    rho_TA[:a,:b] = rho[:a,:b]
    rho_TA[a:,b:] = rho[a:,b:]
    rho_TA[a:,:b] = rho[a:,:b].T
    rho_TA[:a,b:] = rho[:a,b:].T
    # v - eigenvectors, w - eigenvalues
    w, v = LA.eig(rho_TA)
    # PPT Criterion: Are all eigenvalues >=0?
    if all(i >= 0 for i in w):
        # print('Yes: separable state.')
        ppt = 0
    else:
        # print('No: entangled state.')
        ppt = 1
    return w,v,ppt

In [6]:
# Calculating the limit of entanglement

r = 0.5
eps = 0.5
ppa = 0

i = 0

while eps >= 10**(-15) and r>=0 and r<=1 and i<=10**5:
    rho_ent = r*rho_AB+(1-r)*rho_I
    w,v, ppt = Ent_cert(rho_ent)

    eps = eps/2

    if ppt == 0:
        r = r + eps
    else:
        r = r - eps

    i += 1

eta_ent = r

print(eta_ent)

0.33333333333333304


## Bell's non-locality 

Bell scenarios are determined by the following finite values:
- the number of parts;
- the number of possible measurements of each part;
- the number of possible outcomes of each possible measurement of each
part.

### Lower bound: SDP or Outer polytope approximation

### Upper bound: Bell's inequalities - SewSaw

CHSH
$$\langle A_0 B_0\rangle+\langle A_1 B_0\rangle+\langle A_0 B_1\rangle−\langle A_1 B_1\rangle\leq 2$$

I3322
$$−\langle A_0\rangle−\langle A_1\rangle−\langle B_0\rangle−\langle B_1\rangle−\langle A_0 B_0\rangle−\langle A_1 B_0\rangle−\langle A_2 B_0\rangle−\langle A_0 B_1\rangle−\langle A_1 B_1\rangle−\langle A_2 B_1\rangle−\langle A_0 B_2\rangle+\langle A_1 B_2\rangle\leq 4$$

## Desigualdades de Bell
Seja $\vec{p}\in\mathcal{P}_Q$, de forma que 
$$p(ab|xy) = \text{tr}[(E_{a|x}\otimes F_{b|y})\rho],$$
onde $\rho$ é o estado do sistema, e $\{ E_{a|x}\}$ e $\{ F_{b|y}\}$ são os POVMs de Alice e Bob, respectivamente.

Neste contexto, uma desigualdade de Bell pode ser escrita como
$$D(\vec{p}) = c^T\vec{p}=\text{tr}[G\rho]\leq d_L$$
onde
$$G = \sum_{a,b,x,y}c(a,b,x,y)(E_{a|x}\otimes F_{b|y})$$
é um observável associado à desigualdade.

## Algoritmo See-Saw
O seguinte algoritmo iterativo converge para uma cota inferior da máxima violação quântica de uma desigualdade de Bell:
1. Dadas dimensões locais $d_A$ e $d_B$, sorteie POVMs $\{ E_{a|x}\}$ atuando em $\mathbb{C}^{d_A}$ e $\{ F_{b|y}\}$ atuando em $\mathbb{C}^{d_B}$;
2. Construa o observável $G$ associado à desigualdade, obtenha o autovetor $|\psi\rangle$ associado ao maior autovalor, e faça $\rho = |\psi\rangle\langle\psi|$;
3. Maximize $D(\rho, \{ E_{a|x}\}, \{ F_{b|y}\})$ sobre $\{ E_{a|x}\}$, mantendo $\rho$ e $\{ F_{b|y}\}$ fixos;
4. Maximize $D(\rho, \{ E_{a|x}\}, \{ F_{b|y}\})$ sobre $\{ F_{b|y}\}$, mantendo $\rho$ e $\{ E_{a|x}\}$ fixos;
5. Volte ao passo 2. e repita até convergência.
O algoritmo see-saw converge para um máximo local de $D(\rho, \{ E_{a|x}\}, \{ F_{b|y}\})$; é recomendável, portanto, tomar vários pontos iniciais distintos.

## Fixando o cenário
Iremos incrementar a dimensão ao longo do projeto, entretanto teremos $d_A = d_B = d$. As medições precisam respeitar as propriedades de POVM
$$M_{a|x}\geq0,\quad\sum_a M_{a|x}=\mathbb{I} \text{ e}\quad M_{a|x}^\dagger = M_{a|x}.$$
Sabemos que uma matrix mais a sua matrix hermitiana é uma matrix hermitiana. Assim, pra garantir a última propriedade, podemos criar uma matrix aleatória complexa $A$ e calcular a nossa medição $M_{a|x} = A + A.\texttt{conj}().\texttt{T}$.

Para garantir a segunda propriedade, podemos fixar o nosso cenário para medições dicotômicas, assim, encontrado $M_{a|x}$, fazemos $M_{a'|x} = \mathbb{I} - M_{a|x}$.

Para garantir a primeira propriedade, devemos calcular o traço da matriz e ele deve ser não-negativo $\text{tr}[M_{a|x}]\geq0$.

In [24]:
# Criando as medições
def measurement(dimension):
    valid = False
    while valid == False:
        A = np.random.rand(dimension,dimension) + (1j)*(np.random.rand(dimension,dimension))
        M_a = A + A.conj().T
        M_b = np.eye(dimension) - M_a
        trace_a = np.real(np.trace(M_a))
        trace_b = np.real(np.trace(M_b))
        if trace_a>=0 and trace_b>=0:
            valid = True
        else: 
            valid = False
    return M_a, M_b
M_a, M_b = measurement(2)
print(M_a)
print(M_b)

[[1.35198485+0.j         1.21258306-0.68277862j]
 [1.21258306+0.68277862j 0.0093399 +0.j        ]]
[[-0.35198485+0.j         -1.21258306+0.68277862j]
 [-1.21258306-0.68277862j  0.9906601 +0.j        ]]


## Construindo a desigualdade de Bell
Já fixamos a quantidade de resultados por medição, agora vamos fixar em duas a quantidade de medições por parte. Sendo assim, agora temos
$$\{ E_{a|x}\} = [E_{\pm|x_1}, E_{\pm|x_2}]\quad\text{e}\quad\{ F_{b|y}\}= [F_{\pm|y_1}, F_{\pm|y_2}].$$
O observável $G$ associado à desigualdade CHSH é dado por 
$$G_{\text{CHSH}} = E_{x_1}\otimes F_{y_1}+E_{x_2}\otimes F_{y_1}+E_{x_1}\otimes F_{y_2}-E_{x_2}\otimes F_{y_2}.$$
Para obtermos o autovetor $|\psi\rangle$ associado ao maior autovalor, e recuperarmos $\rho = |\psi\rangle\langle\psi|$, precisamos resolver o problema de otimização:
$$\begin{align*}
\text{given} \quad & G \quad (c,\{E_{a|x}\},\{F_{b|y}\})\\
\max \quad &\text{tr}[G\rho] = \sum_{a,b,x,y}c(a,b,x,y)\text{tr}[(E_{a|x}\otimes F_{b|y})\rho]\\
\text{subject to}\quad & \text{tr}[\rho] = 1 \quad \text{Normalização} \\
\quad & \text{tr}[\rho^2] = 1 \quad \text{Estado puro}
\end{align*}
$$

In [ ]:
# Otimização do estado
def max_rho(dimension, c, measurements_A, measurements_B):

    #Creating the problem
    P = pic.Problem()

    #Creating the optimization variables

    rho = pic.HermitianVariable('rho',(dimension,dimension))

    #Creating the constraints
    P.add_constraint(pic.trace(rho)==1)
    P.add_constraint(pic.trace(rho**2)==1)

    trace = [pic.trace((measurements_A[i]@measurements_B[j])*rho) for i in range(k*m) for j in range(k*m)]

    D = pic.sum([c[i]*trace[i] for i in range(k*m)])

    #Setting the objective
    P.set_objective('max',D)

    #Finding the solution
    P.solve()

    return rho

rho = max_rho(4,)

In [34]:
k = 2
m = 2
print([i*j for i in range(k*m) for j in range(k*m)])

[0, 0, 0, 0, 0, 1, 2, 3, 0, 2, 4, 6, 0, 3, 6, 9]


## Otimização sobre as medições de cada parte
Queremos maximizar $D(\rho, \{ E_{a|x}\}, \{ F_{b|y}\})$ sobre $\{ E_{a|x}\}$, mantendo $\rho$ e $\{ F_{b|y}\}$ fixos. Para isso vamos realizar o seguinte protocolo de otimização
$$\begin{align*}
\text{given} \quad & c,\rho,\{F_{b|y}\}\\
\max \quad & \sum_{a,b,x,y}c(a,b,x,y)\text{tr}[(E_{a|x}\otimes F_{b|y})\rho]\\
\text{subject to}\quad & E_{a|x}\geq0,\quad\forall a,x \\
\quad & \sum_a E_{a|x} = \mathbb{I}, \quad \forall x
\end{align*}
$$

In [ ]:
# Otimização das medições
def max_Measurements_Alice(dimension,c,rho, measurements_B,k,m):

    #Creating the problem
    P = pic.Problem()

    #Creating the optimization variables

    M_A = [pic.HermitianVariable('Measurements_Alice[{}]'.format(i),(dimension,dimension)) for i in range(k*m)]

    #Creating the constraints
    P.add_list_of_constraints([M_A[i]>=0 for i in range(k**m)])

    ident = [pic.sum([M_A[i+j] for i in range(k)]) for j in range(m)]

    P.add_list_of_constraints([ident[j]==np.eye(dimension,dtype='complex') for j in range(m)]) 

    G = 

    D = pic.trace()

    #Setting the objective
    P.set_objective('max',D)

    #Finding the solution
    P.solve()

    return M_A

def max_Measurements_Bob(dimension,c,rho, measurements_A,k,m):

    #Creating the problem
    P = pic.Problem()

    #Creating the optimization variables

    rho = pic.HermitianVariable('rho',(dimension,dimension))

    #Creating the constraints
    P.add_constraint(pic.trace(rho)==1)
    P.add_constraint(pic.trace(rho**2)==1)

    G = 

    D = pic.trace()

    #Setting the objective
    P.set_objective('max',D)

    #Finding the solution
    P.solve()

    return M_B